In [77]:
import numpy as np
import pandas as pd
import pyarrow as pa
import matplotlib.pyplot as plt
from modules import etl

In [78]:
data = pd.read_csv("data/data_project_metset.csv", sep=";", dtype_backend='pyarrow', engine='pyarrow')
codebook = pd.read_json("data/codebook.json", dtype_backend='pyarrow')
pa_string = pd.ArrowDtype(pa.string())
nan = [0, -66, -77, -99]

In [79]:
etl = etl.Etl(codebook=codebook, data=data)

In [80]:
print(data.dtypes)
print('--------')
print(codebook.dtypes)


lfdn             int64[pyarrow]
external_lfdn    int64[pyarrow]
tester           int64[pyarrow]
dispcode         int64[pyarrow]
lastpage         int64[pyarrow]
                      ...      
rts7288986       int64[pyarrow]
rts7288987       int64[pyarrow]
rts7288988       int64[pyarrow]
rts7288989       int64[pyarrow]
rts7288990       int64[pyarrow]
Length: 283, dtype: object
--------
group_id         string[pyarrow]
main_question    string[pyarrow]
items                     object
options                   object
dtype: object


In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Columns: 283 entries, lfdn to rts7288990
dtypes: int64[pyarrow](267), string[pyarrow](16)
memory usage: 2.5 MB


In [82]:
data.head(10)

,lfdn,external_lfdn,tester,dispcode,lastpage,quality,duration,p_0001,browser,referer,...,rts7288979,rts7288980,rts7288981,rts7288982,rts7288983,rts7288986,rts7288987,rts7288988,rts7288989,rts7288990
0,5,0,0,31,7288991,-77,456,testJZ3,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,https://norstatsurveys.com/,...,341,353,364,380,412,418,428,440,452,456
1,7,0,0,31,7288991,-77,1717,1199209,Mozilla/5.0 (Linux; Android 13; SM-G998B) Appl...,https://norstatsurveys.com/,...,1055,1100,1154,1218,1349,1412,1485,1615,1710,1717
2,8,0,0,31,7288991,-77,598,101222,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,-99,...,344,357,380,426,479,495,526,569,596,598
3,9,0,0,32,7288991,-77,-1,100212,Mozilla/5.0 (Linux; Android 13; SAMSUNG SM-G99...,https://norstatsurveys.com/,...,4023,4036,4081,4134,4206,4246,4300,4363,4394,4397
4,11,0,0,31,7288991,-77,1127,1184194,Mozilla/5.0 (Linux; Android 11; SAMSUNG SM-A32...,https://norstatsurveys.com/,...,542,562,623,687,823,909,1011,1096,1122,1127
5,12,0,0,31,7288991,-77,508,101828,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,https://norstatsurveys.com/,...,230,239,267,304,354,376,421,464,504,508
6,13,0,0,31,7288991,-77,1748,102939,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,https://norstatsurveys.com/,...,1200,1216,1275,1313,1452,1522,1664,1705,1745,1748
7,16,0,0,31,7288991,-77,1367,1193203,Mozilla/5.0 (Linux; Android 9; ANE-LX1) AppleW...,https://norstatsurveys.com/,...,907,942,987,1049,1142,1198,1253,1336,1362,1367
8,17,0,0,31,7288991,-77,732,1171181,Mozilla/5.0 (Linux; Android 12; M2101K7BNY) Ap...,https://norstatsurveys.com/,...,408,422,462,519,594,623,648,702,726,732
9,18,0,0,31,7288991,-77,886,103040,Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like M...,https://norstatsurveys.com/,...,528,549,591,641,707,746,810,866,883,886


In [83]:
df = data.copy()

In [ ]:
# Drop rts - no encoding information
columns_to_drop = df.filter(regex=r'^rts\d+').columns

df = df.drop(columns=columns_to_drop)

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Columns: 260 entries, lfdn to v_2920
dtypes: int64[pyarrow](244), string[pyarrow](16)
memory usage: 2.3 MB


In [86]:
df.head(10)

,lfdn,external_lfdn,tester,dispcode,lastpage,quality,duration,p_0001,browser,referer,...,v_2933,v_1658,v_1659,v_1660,v_1661,v_1662,v_1663,v_1664,v_1665,v_2920
0,5,0,0,31,7288991,-77,456,testJZ3,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,https://norstatsurveys.com/,...,5,2,4,4,1,5,3,5,3,test
1,7,0,0,31,7288991,-77,1717,1199209,Mozilla/5.0 (Linux; Android 13; SM-G998B) Appl...,https://norstatsurveys.com/,...,3,2,2,5,2,1,3,3,2,-99
2,8,0,0,31,7288991,-77,598,101222,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,-99,...,5,3,1,4,3,2,1,3,3,-99
3,9,0,0,32,7288991,-77,-1,100212,Mozilla/5.0 (Linux; Android 13; SAMSUNG SM-G99...,https://norstatsurveys.com/,...,3,1,1,5,4,1,1,4,2,-99
4,11,0,0,31,7288991,-77,1127,1184194,Mozilla/5.0 (Linux; Android 11; SAMSUNG SM-A32...,https://norstatsurveys.com/,...,5,2,2,5,4,2,1,4,1,-99
5,12,0,0,31,7288991,-77,508,101828,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,https://norstatsurveys.com/,...,5,2,1,5,5,2,1,4,1,-99
6,13,0,0,31,7288991,-77,1748,102939,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,https://norstatsurveys.com/,...,2,1,2,5,5,1,1,5,1,-99
7,16,0,0,31,7288991,-77,1367,1193203,Mozilla/5.0 (Linux; Android 9; ANE-LX1) AppleW...,https://norstatsurveys.com/,...,3,3,1,5,4,1,2,5,3,-99
8,17,0,0,31,7288991,-77,732,1171181,Mozilla/5.0 (Linux; Android 12; M2101K7BNY) Ap...,https://norstatsurveys.com/,...,3,3,2,3,3,2,2,4,2,-99
9,18,0,0,31,7288991,-77,886,103040,Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like M...,https://norstatsurveys.com/,...,4,1,1,5,5,1,1,5,1,-99


In [87]:
df.lastpage.agg([
    'nunique'
])

nunique    1
Name: lastpage, dtype: int64

In [88]:
df.agg([
    'nunique'
])

,lfdn,external_lfdn,tester,dispcode,lastpage,quality,duration,p_0001,browser,referer,...,v_2933,v_1658,v_1659,v_1660,v_1661,v_1662,v_1663,v_1664,v_1665,v_2920
nunique,960,1,1,2,1,1,702,957,482,23,...,5,5,5,5,5,5,5,5,5,120


In [ ]:
# Drop columns with single realization
cols_to_drop = df.columns[df.nunique() == 1]
df = df.drop(columns=cols_to_drop)
print(f"Dropped columns: {list(cols_to_drop)}")

Dropped columns: ['external_lfdn', 'tester', 'lastpage', 'quality', 'device_type', 'quota_rejected_id', 'output_mode', 'javascript', 'flash', 'language', 'cleaned', 'date_of_first_mail', 'v_2889', 'v_2890', 'v_2891', 'v_2892', 'v_1754']


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Columns: 243 entries, lfdn to v_2920
dtypes: int64[pyarrow](228), string[pyarrow](15)
memory usage: 2.2 MB


In [101]:
# Replace coded missing values with NA
df = df.replace(nan, pd.NA)

In [102]:
df.isna().sum()

lfdn        0
dispcode    0
duration    0
p_0001      0
browser     0
           ..
v_1662      0
v_1663      0
v_1664      0
v_1665      0
v_2920      1
Length: 243, dtype: int64

In [112]:
df_categorial = etl.get_item_type == 'ordinal'

In [113]:
df_categorial

False